In [1]:
import streamlit as st
import os
import requests
from utils.save_recipes import save_recipe_as_pdf, get_recipe_pdf_download_link
from dotenv import load_dotenv
load_dotenv()
import openai
from google.oauth2 import service_account
from spellchecker import SpellChecker
# Set your API key
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])

In [1]:
import pandas as pd
df = pd.read_csv('./inventory.csv')
df.head()

,Name,Quantity,Volume per Unit (ml),Cost per Unit,Cost per ml,Cost per oz
0,Vodka,6,750,51.758815,0.069012,2.040919
1,Gin,2,1000,20.383485,0.020383,0.602811
2,Rum,1,1000,75.476875,0.075477,2.232115
3,Tequila,9,750,43.807963,0.058411,1.727406
4,Whiskey,5,750,90.563148,0.120751,3.571026


In [3]:
prices_df = df[['Name', 'Volume per Unit (ml)', 'Cost per Unit']]
prices_df.to_csv('prices.csv', index=False)

In [5]:
prices_df['Cost per Unit'] = avg_prices = [
    22.5,  # Vodka
    30,    # Gin
    22.5,  # Rum
    35,    # Tequila
    37.5,  # Whiskey
    30,    # Brandy
    45,    # Scotch
    35,    # Bourbon
    45,    # Cognac
    45,    # Absinthe
    22.5,  # Sake
    37.5,  # Rye
    45,    # Mezcal
    22.5,  # Amaretto
    22.5,  # Baileys
    37.5,  # Campari
    37.5,  # Cointreau
    30,    # Frangelico
    40,    # Grand Marnier
    30,    # Jägermeister
    22.5,  # Kahlúa
    27.5,  # Limoncello
    30,    # Sambuca
    22.5,  # Southern Comfort
    30,    # Tia Maria
    15,    # Triple Sec
    30,    # Pisco
    20,    # Port
    17.5,  # Sherry
    20,    # Vermouth
    30,    # Aperol
    15,    # Bitters
    50,    # Chartreuse
    40,    # Chambord
    35,    # Fernet Branca
    37.5,  # Galliano
    30,    # Goldschlager
    15,    # Malibu
    22.5,  # Midori
    15,    # Peach Schnapps
    22.5,  # Peppermint Schnapps
    30,    # Sloe Gin
    40,    # St Germain
    22.5,  # Umeshu
    27.5,  # Anisette
    37.5,  # Aquavit
    30,     # Becherovka
    30,    # Bénédictine
    22.5,  # Crème de Cacao
    30     # Crème de Menthe
]


C:\Users\sjufa\AppData\Local\Temp\ipykernel_18684\604942063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['Cost per Unit'] = avg_prices = [


In [6]:
prices_df['Cost per Unit'] = prices_df['Cost per Unit'].astype(float) * .45
prices_df['Cost per Unit'] = prices_df['Cost per Unit'].round(2)
prices_df.head()

C:\Users\sjufa\AppData\Local\Temp\ipykernel_18684\2673601265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['Cost per Unit'] = prices_df['Cost per Unit'].astype(float) * .45
C:\Users\sjufa\AppData\Local\Temp\ipykernel_18684\2673601265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['Cost per Unit'] = prices_df['Cost per Unit'].round(2)


,Name,Volume per Unit (ml),Cost per Unit
0,Vodka,750,10.12
1,Gin,1000,13.50
2,Rum,1000,10.12
3,Tequila,750,15.75
4,Whiskey,750,16.88


In [8]:
df['Cost per Unit'] = prices_df['Cost per Unit']
df.head()

# Drop the "Cost per ml" and "Cost per oz" columns
df.drop(['Cost per ml', 'Cost per oz'], axis=1, inplace=True)

In [9]:
# Create a "Cost per ml" column that is the cost per unit divided by the volume per unit
df['Cost per ml'] = df['Cost per Unit'] / df['Volume per Unit (ml)']

# Create a cost per oz column that is the cost per ml multiplied by 29.5735
df['Cost per oz'] = df['Cost per ml'] * 29.5735

# Round the cost per oz column to 2 decimal places
df['Cost per oz'] = df['Cost per oz'].round(2)

df.head()

,Name,Quantity,Volume per Unit (ml),Cost per Unit,Cost per ml,Cost per oz
0,Vodka,6,750,10.12,0.013493,0.40
1,Gin,2,1000,13.50,0.013500,0.40
2,Rum,1,1000,10.12,0.010120,0.30
3,Tequila,9,750,15.75,0.021000,0.62
4,Whiskey,5,750,16.88,0.022507,0.67


In [10]:
df.to_csv('inventory.csv', index=False)

In [2]:
from google.cloud import vision

def detect_document(uploaded_image):

    client = vision.ImageAnnotatorClient(credentials=credentials)

    #with io.BytesIO(uploaded_image) as image_file:
    #    content = image_file

    #image = vision.Image(uploaded_image)

    response = client.document_text_detection(image=uploaded_image)

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))
    
    response_text = response.full_text_annotation.text


    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    
    return response_text